In [13]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import sqlite3 as db

In [14]:
# Load data
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")

# Add column headers
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

# Create sqladb
sqladb = db.connect('/Users/suruchi/Desktop/Personal/Aditya/Personal/Data Science/Acadgild/Assignment/12/sqladb')

#Creating database cursor for further querying the data
cursor = sqladb.cursor()

# Create table and load data
df.to_sql('Adult', sqladb, if_exists='replace', index=False)

In [15]:
# 1. Select 10 records from the adult sqladb
print("1. Select 10 records from the adult sqladb")
query = "SELECT * FROM Adult limit 10"
print(pd.read_sql_query(query,sqladb))

1. Select 10 records from the adult sqladb
   age          workclass  fnlwgt      education  education_num  \
0   50   Self-emp-not-inc   83311      Bachelors             13   
1   38            Private  215646        HS-grad              9   
2   53            Private  234721           11th              7   
3   28            Private  338409      Bachelors             13   
4   37            Private  284582        Masters             14   
5   49            Private  160187            9th              5   
6   52   Self-emp-not-inc  209642        HS-grad              9   
7   31            Private   45781        Masters             14   
8   42            Private  159449      Bachelors             13   
9   37            Private  280464   Some-college             10   

           marital_status          occupation    relationship    race  \
0      Married-civ-spouse     Exec-managerial         Husband   White   
1                Divorced   Handlers-cleaners   Not-in-family   White   


In [16]:
# 2. Show me the average hours per week of all men who are working in private sector
print("\n2. Show me the average hours per week of all men who are working in private sector")
query = "SELECT sex,workclass, avg(hours_per_week) AverageHoursPerWeek FROM Adult where sex = 'Male' and workclass = 'Private'  group by sex,workclass"
print(pd.read_sql_query(query, sqladb))


2. Show me the average hours per week of all men who are working in private sector
Empty DataFrame
Columns: [sex, workclass, AverageHoursPerWeek]
Index: []


In [17]:
# 3. Show me the frequency table for education, occupation and relationship, separately
print("\n3. Show me the frequency table for education, occupation and relationship, separately")
print("\nFrequency table for education")
query = "SELECT  education, count(education) frequency from Adult group by education order by frequency desc "
print(pd.read_sql_query(query,sqladb))


3. Show me the frequency table for education, occupation and relationship, separately

Frequency table for education
        education  frequency
0         HS-grad      10501
1    Some-college       7291
2       Bachelors       5354
3         Masters       1723
4       Assoc-voc       1382
5            11th       1175
6      Assoc-acdm       1067
7            10th        933
8         7th-8th        646
9     Prof-school        576
10            9th        514
11           12th        433
12      Doctorate        413
13        5th-6th        333
14        1st-4th        168
15      Preschool         51


In [18]:
print("\nFrequency table for occupation")
query = "SELECT  occupation, count(occupation) frequency from Adult group by occupation order by frequency desc "
print(pd.read_sql_query(query,sqladb))

print("\nFrequency table for relationship")
query = "SELECT  relationship, count(relationship) frequency from Adult group by relationship order by frequency desc "
print(pd.read_sql_query(query,sqladb))


Frequency table for occupation
            occupation  frequency
0       Prof-specialty       4140
1         Craft-repair       4099
2      Exec-managerial       4066
3         Adm-clerical       3769
4                Sales       3650
5        Other-service       3295
6    Machine-op-inspct       2002
7                    ?       1843
8     Transport-moving       1597
9    Handlers-cleaners       1370
10     Farming-fishing        994
11        Tech-support        928
12     Protective-serv        649
13     Priv-house-serv        149
14        Armed-Forces          9

Frequency table for relationship
      relationship  frequency
0          Husband      13193
1    Not-in-family       8304
2        Own-child       5068
3        Unmarried       3446
4             Wife       1568
5   Other-relative        981


In [19]:
# 4. Are there any people who are married, working in private sector and having a masters degree
print("\n4. Are there any people who are married, working in private sector and having a masters degree")
query = """SELECT count(*) as People FROM Adult WHERE "marital_status" like "Married-%" AND workclass="Private" AND education="Masters" """
print(pd.read_sql_query(query,sqladb))


4. Are there any people who are married, working in private sector and having a masters degree
   People
0       0


In [20]:
# 5. What is the average, minimum and maximum age group for people working in different sectors
print("\n5. What is the average, minimum and maximum age group for people working in different sectors")
query = """SELECT workclass, avg(age) average, min(age) minimum, max(age) maximum FROM Adult GROUP BY workclass"""
print(pd.read_sql_query(query,sqladb))


5. What is the average, minimum and maximum age group for people working in different sectors
           workclass    average  minimum  maximum
0                  ?  40.960240       17       90
1        Federal-gov  42.590625       17       90
2          Local-gov  41.751075       17       90
3       Never-worked  20.571429       17       30
4            Private  36.797585       17       90
5       Self-emp-inc  46.017025       17       84
6   Self-emp-not-inc  44.969697       17       90
7          State-gov  39.436392       17       81
8        Without-pay  47.785714       19       72


In [21]:
# 6. Calculate age distribution by country
print("\n6. Calculate age distribution by country")
query = """SELECT native_country, Round(Round((count(age)*100),5)/Round((select COUNT(age) from Adult),5),5) as "Age Distribution"  FROM Adult group by native_country order by count(age) desc """
print(pd.read_sql_query(query,sqladb))


6. Calculate age distribution by country
                 native_country  Age Distribution
0                 United-States          89.58538
1                        Mexico           1.97482
2                             ?           1.79054
3                   Philippines           0.60811
4                       Germany           0.42076
5                        Canada           0.37162
6                   Puerto-Rico           0.35012
7                   El-Salvador           0.32555
8                         India           0.30713
9                          Cuba           0.29177
10                      England           0.27641
11                      Jamaica           0.24877
12                        South           0.24570
13                        China           0.23034
14                        Italy           0.22420
15           Dominican-Republic           0.21499
16                      Vietnam           0.20577
17                    Guatemala           0.19656
18      

In [22]:
# 7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'
print("\n7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'")
query = """SELECT capital_gain as 'capital-gain', capital_loss as 'capital-loss', (capital_gain - capital_loss) as 'Net-Capital-Gain' FROM Adult """
print(pd.read_sql_query(query,sqladb))


7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'
       capital-gain  capital-loss  Net-Capital-Gain
0                 0             0                 0
1                 0             0                 0
2                 0             0                 0
3                 0             0                 0
4                 0             0                 0
5                 0             0                 0
6                 0             0                 0
7             14084             0             14084
8              5178             0              5178
9                 0             0                 0
10                0             0                 0
11                0             0                 0
12                0             0                 0
13                0             0                 0
14                0             0                 0
15                0             0                 0
16            

In [23]:
sqladb.close()